In [23]:
import os
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
import numpy as np
import re
from datetime import date
from datetime import datetime
import matplotlib.pylab as plt
plt.rcdefaults()

#IMPORT AND EXTRACT DATABASE:

df = sqlite3.connect('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/raw/Miguel318.db')
dfBI = pd.read_sql_query("SELECT * FROM business_info", df) 
dfRI = pd.read_sql_query("SELECT * FROM rank_info", df)
dfPI = pd.read_sql_query("SELECT * FROM personal_info", df)

#WRANGLING - DATA CLEANING - DATABASE:

dfBI.drop(['realTimeWorth'], axis='columns', inplace=True)
dfBI.drop(['Unnamed: 0'], axis='columns', inplace=True)
dfBI.drop(['realTimePosition'], axis='columns', inplace=True)
dfBI.replace({"BUSD": ' '})
dfBI.worthChange.replace('na', '0.0', regex=True, inplace=True)
dfBI.worth.replace('[BUSD]+$', '', regex=True, inplace=True)        
dfBI.worthChange.replace('[millions USD]+$', '', regex=True, inplace=True)
dfBI['worth'] = dfBI['worth'].astype('float').astype('int')
dfBI['worthChange'] = dfBI['worthChange'].astype('float').astype('int')
dfBI['id'] = dfBI['id'].astype('object').astype('int')
dfBI = dfBI.rename(columns={'worth':'worth BUSD', 'worthChange':' worthChange millions USD'})
dfBI = dfBI.drop_duplicates()
dfBI = dfBI.reset_index(drop=True)
dfRI.drop(['Unnamed: 0'], axis='columns', inplace=True)
dfRI['position'] = dfRI['position'].fillna(0)   
dfRI['position'] = dfRI['position'].astype('float').astype('int')
dfRI = dfRI.apply(lambda x: x.astype(str).str.lower())
dfRI['position'] = dfRI['position'].astype('object').astype('int')
dfRI['id'] = dfRI['id'].astype('object').astype('int')
dfRI = dfRI.drop(dfRI[dfRI['position']==0].index)
dfRI = dfRI.reset_index(drop=True)
dfPI.drop(['Unnamed: 0'], axis='columns', inplace=True)
dfPI.drop(['lastName'], axis='columns', inplace=True)
dfPI.gender.replace('[M]+$', 'Male', regex=True, inplace=True)
dfPI.gender.replace('[F]+$', 'Female', regex=True, inplace=True)
dfPI['gender'] = dfPI['gender'].astype('object').astype('str')
dfPI.age.replace('[years old]+$', '', regex=True, inplace=True)   
dfPI['age'] = dfPI['age'].fillna(0)   
dfPI['age'] = dfPI['age'].astype('float').astype('int') 

now = datetime.now()
now.year

stats = dfPI.describe().T
stats['IQR'] = stats['75%'] - stats['25%']


outliers = pd.DataFrame(columns=dfPI.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = dfPI[(dfPI[col] < lower) | 
            (dfPI[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)
    'sort=False'

dfPI['new_age'] = now.year - outliers['age']
dfPI.new_age.fillna(dfPI.age, inplace=True)

dfPI['age'] = dfPI['new_age']
dfPI.drop(['new_age'], axis='columns', inplace=True)
dfPI.age.replace(now.year, 'None', regex=True, inplace=True)
dfPI = dfPI.drop_duplicates()
dfPI = dfPI.reset_index(drop=True)

#EXPORT CLEAN DATA TO LOCAL:
        
dfBI.to_csv('/Users/MIGUEL/Desktop/TestProject/dfBussinesInfoCLEAN.csv', index=False)
dfRI.to_csv('/Users/MIGUEL/Desktop/TestProject/dfrankinfoCLEAN.csv', index=False)
dfPI.to_csv('/Users/MIGUEL/Desktop/TestProject/dfPersonalInfoCLEAN.csv', index=False)

#MERGE DATAFRAMES:

df4 = pd.merge(dfBI, dfRI, on=['id', 'id'])
df_total = pd.merge(df4, dfPI, on=['id', 'id'])

df_total.drop(['id'], axis='columns', inplace=True)

df_total.to_csv('/Users/MIGUEL/Desktop/TestProject/df_total.csv', index=False)

#WEB - SCRAPPING:
    #Extract URL - Scrapper:

url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

table = soup.find_all('table',{'class':'wikitable sortable mw-datatable'})[0]
table

    # tr represent the table rows:
rows = table.find_all('tr')
rows_parsed = [row.text for row in rows]
rows_parsed

    #Parsering:

def smart_parser(row_text):
    row_text = row_text.replace('\nPopulation\n\n\n', '\nPopulation\n').strip('\n')
    row_text = row_text.replace('\n\n', '\n').strip('\n')
    row_text = row_text.replace('\nPopulation\n% of World\n', '\nPopulation % of World\n').strip('\n')
    row_text = re.sub('\[\d\]', '', row_text)
    
    return list(map(lambda x: x.strip(), row_text.split('\n')))

well_parsed = list(map(lambda x: smart_parser(x), rows_parsed))

colnames = well_parsed[0]
data = well_parsed[1:]

    #Creating DF:
df_web_scrapping = pd.DataFrame(data, columns=colnames)

df_web_scrapping['Country(or dependent territory)']=df_web_scrapping['Country(or dependent territory)'].replace(['\[(.*?)\]','\(([^\)]+)\)'], ['',''], regex=True)

df_web_scrapping.columns = ['Rank', 'country','Population',' % of World Population','Date','Source']

#Exporting to CSV: 

df_web_scrapping.to_csv('/Users/MIGUEL/Desktop/TestProject/df_web_scrapping.csv', index=False)

#Import df_web_scrapping, full

df_web_scrapping = pd.read_csv('/Users/MIGUEL/Desktop/TestProject/df_web_scrapping.csv')
df_total = pd.read_csv('/Users/MIGUEL/Desktop/TestProject/df_total.csv')
df_total.country.replace('None', np.nan, regex=True, inplace=True)
df_total.country.isnull().sum()
df_no_null_values = df_total[df_total.age.isnull()]
df_no_null_values.shape
df_total.drop(df_total[df_total.country.isnull()].index, inplace = True)
df_total.country.isnull().sum()

#Merge - df_total,no null values in country col with df_web_scrapping:

df_project2 = pd.merge(df_web_scrapping, df_total, on=['country', 'country'])

#EXPORTING WEB-SCRAPPING-DF_TOTAL TO LOCAL:

df_project2.to_csv('/Users/MIGUEL/Desktop/TestProject/df_project2CLEAN.csv', index=False)

#GRAPHICS - STATISTICS - ANALYTICS:
    #Extracting data:

project1_df = pd.read_csv('/Users/MIGUEL/Desktop/TestProject/df_total.csv')
project2_df = pd.read_csv('/Users/MIGUEL/Desktop/TestProject/df_project2CLEAN.csv')

    #Checking Outliers: 

stats = project1_df.describe().T
stats['IQR'] = stats['75%'] - stats['25%']
stats

outliers = pd.DataFrame(columns=project1_df.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = project1_df[(project1_df[col] < lower) | 
                   (project1_df[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)
    'sort=False'

outliers

#Graphic 1: worth BUSD - full dataframe
graphic_1 = project1_df['worth BUSD'].hist(bins=100, figsize=(16,8))
graphic_1

plt.savefig("worth BUSD deviation.png", bbox_inches='tight')

#Graphic 2: wortChange millions USD - full dataframe
graphic_2 = project1_df[' worthChange millions USD'].hist(bins=10, figsize=(16,8))
graphic_2

plt.savefig("worthChange millions USD", bbox_inches='tight')

#Graphic 3: worth BUSD - outliers dataframe 
graphic_3 = outliers['worth BUSD'].hist(bins=10, figsize=(16,8))
graphic_3

plt.savefig("worth BUSD - outliers", bbox_inches='tight')

#Graphic 4: wortChange millions USD - dataframe outliers
graphic_4 = outliers[' worthChange millions USD'].hist(bins=10, figsize=(16,8))
graphic_4

plt.savefig("worthChange millions USD - outliers", bbox_inches='tight')

#Graphic 5: Scattter position - worth
my_plot = project1_df.plot("position", "worth BUSD", kind="scatter")

plt.savefig("scatter position - worth") 

#Graphic 6: Scatter position - worthChange 
my_plot2 = project1_df.plot("position", " worthChange millions USD", kind="scatter")

plt.savefig("scatter position-worthchange")

#Wealth comparison between genders
    #Stadistics

wealth_comparison = project1_df.groupby(['gender','worth BUSD']).agg(['count', 'mean'])
wealth_comparison

    #Graphic7

fig = plt.figure(figsize=(30,10))
project1_df.gender[project1_df.position].value_counts(normalize = True).plot(kind='barh', alpha=0.5)
plt.title('Wealth comparison between genders')

plt.savefig("Wealth comparison between genders") 

#Wealth comparison between countries 
    #Stadistics

wealth_comparison = project2_df.groupby(['country','worth BUSD']).agg(['count', 'mean'])

    #Graphic8

fig = plt.figure(figsize=(50,50))
project2_df.country[project1_df.position].value_counts(normalize = True).plot(kind='barh', alpha=0.5)
plt.title('Wealth comparison between countries')

plt.savefig("Wealth comparison between countries") 

#Outliers - Project2_df

stats2 = project2_df.describe().T
stats['IQR'] = stats['75%'] - stats['25%']
stats2

outliers2 = pd.DataFrame(columns=project1_df.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = project2_df[(project2_df[col] < lower) | 
                   (project2_df[col] > upper)].copy()
    results['Outliers2'] = col
    outliers2 = outliers2.append(results)
    'sort=False'

outliers2

#Graphic9: 

my_plot2 = project2_df.plot("position", " worthChange millions USD", kind="scatter")

plt.savefig("scatter-outliers-scrapping")
print('pollas')

                          worthChange millions USD      position             
                                             count mean    count         mean
country       worth BUSD                                                     
Algeria       4                                  1  0.0        1   563.000000
Angola        2                                  1  0.0        1   933.000000
Argentina     1                                  1  0.0        1  1378.000000
              3                                  1  0.0        1   796.000000
Australia     1                                 11  0.0       11  1815.727273
...                                            ...  ...      ...          ...
United States 21                                 1  0.0        1    44.000000
              23                                 1  0.0        1    34.000000
              84                                 1  0.0        1     3.000000
              90                                 1  0.0        1